In [23]:
import opentrons_json as oj
from opentrons_json import plates,commands
import uuid
from jsonschema import validate
import json
with open('opentrons_json/protocol-schema.json') as f:
    data = json.load(f)
import pandas as pd

In [48]:
pipette = str(uuid.uuid4())
tip_box = "tip_box"

protocol = {
    "protocol-schema": "1.0.0",

  "metadata": {
    "protocol-name": "Example transfer protocol",
    "author": "Keoni Gandall",
  },
    "default-values": oj.default_values,
    "designer-application": oj.designer_application,
    "robot": oj.ot2_standard,
    "pipettes": {pipette:oj.pipette("left", "p300_single", "p300_single_v1.3")},
    "labware": {
        tip_box: oj.labware('10',"opentrons-tiprack-300ul"),
    },
    "procedure": [
        {"subprocedure": [
        oj.commands.aspirate(pipette,50,plate_1,'A1'),
        oj.commands.dispense(pipette,50,plate_2,'A2')]}]
}

In [57]:
options = ['glycerol-plate_1',
 'glycerol-plate_10',
 'glycerol-plate_11',
 'glycerol-plate_12',
 'glycerol-plate_13',
 'glycerol-plate_15',
 'glycerol-plate_16',
 'glycerol-plate_17',
 'glycerol-plate_18',
 'glycerol-plate_19',
 'glycerol-plate_2',
 'glycerol-plate_20',
 'glycerol-plate_21',
 'glycerol-plate_22',
 'glycerol-plate_23',
 'glycerol-plate_24',
 'glycerol-plate_3',
 'glycerol-plate_4',
 'glycerol-plate_5',
 'glycerol-plate_6',
 'glycerol-plate_8']

collections_loc = pd.read_csv('./../../analysis/collections_loc.csv')[['gene_id','organism','plate_location','address','target','target_plate']]

for option in options:
    desired_plate = collections_loc[collections_loc['plate_location'] == option]
    list_of_target_plates = list(set(desired_plate['target_plate'].tolist()))
    
    new_protocol=json.loads(json.dumps(protocol)) # set new protocol instance
    commands = [] # set new commands
    
    new_protocol['labware'][option] = oj.labware('11','96-flat')
    
    for index,target_plate in enumerate(list_of_target_plates):
        new_protocol['labware'][target_plate] = oj.labware(str(index+1),'96-flat')
        desired_scope = desired_plate[desired_plate['target_plate'] == target_plate]
        
        for index,row in desired_scope.iterrows():
            commands.append(oj.commands.aspirate(pipette,50,option,row['address']))
            commands.append(oj.commands.dispense(pipette,50,target_plate,plates.standard96.locations[int(row['target'])]))
    
    new_protocol['procedure'] = [{"subprocedure": commands}]
    validate(instance=new_protocol,schema=data)
    with open('/home/koeng/gits/opentrons_json/freegenes_move/{}.json'.format(option), 'w') as fp:
        json.dump(new_protocol, fp)